In [1]:
import pandas as pd

school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_merged = pd.merge(student_data, school_data, how="left", on=["school_name","school_name"])

school_data_merged.head(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


In [2]:
# school_data_merged.columns
# school_data_merged = student_data_merged.rename(columns={"student_name":"Student Name",
#                                                         "school_name":"School Name",
#                                                         "reading_score": "Reading Score",
#                                                         "math_score": "Math Score"})

In [6]:
schools = school_data_merged["school_name"].unique()
total_schools = len(schools)
print(total_schools)

total_students = school_data_merged["student_name"].count()
print(total_students)

budget = school_data_merged["budget"].unique()
total_budget = budget.sum()
print(total_budget)

average_math = school_data_merged["math_score"].mean()
print(average_math)

average_reading = school_data_merged["reading_score"].mean()
print(average_reading)

math_score_int = school_data_merged.loc[:,"math_score"].astype(int)

students_passing_math = 0
for i in range(len(math_score_int)):
    if math_score_int[i] >= 70:
        students_passing_math +=1

percentage_passing_math = (students_passing_math/total_students) * 100
print(percentage_passing_math)

reading_score_int = school_data_merged.loc[:,"reading_score"].astype(int)

students_passing_reading = 0

for i in range(len(reading_score_int)):
    if reading_score_int[i] >= 70:
        students_passing_reading +=1

percentage_passing_reading = (students_passing_reading/total_students) * 100
print(percentage_passing_reading)

math_and_reading = school_data_merged.loc[:,"math_score"].astype(int) | school_data_merged.loc[:,"reading_score"].astype(int)
print(math_and_reading)

15
39170
24649428
78.98537145774827
81.87784018381414
74.9808526933878
85.80546336482001
0         79
1        127
2        126
3        123
4        117
        ... 
39165    123
39166     95
39167     93
39168    123
39169     95
Length: 39170, dtype: int32


In [4]:
# for i in range(len(a)):
#     if a[i] >= 70:
#         students_passing_both += 1
#     print(students_passing_both)

# math_and_reading = school_data_merged.loc[(school_data_merged["math_score"].astype(int) 
#  | school_data_merged.loc[:,"reading_score"].astype(int)
# #print(math_and_reading)

#only_billy_and_peter = df.loc[(df["first_name"] == "Billy") | (df["first_name"] == "Peter"), :]
# students_passing_math = school_data_merged["math_score"]
# students_passing_math = school_data_merged.loc[(school_data_merged[:,"math_score"].astype[int] >= 70)]
# print(students_passing_math)
# converted_ufo_df["duration (seconds)"] = converted_ufo_df.loc[:, "duration (seconds)"].astype(float)